I have to find a combination of different types of soups from https://en.wikipedia.org/wiki/Table_of_food_nutrients that covers the nutritional needs below while **minimizing the amount of saturated fat**:


*   At least 2,000 Calories
*   At least 50 grams of Protein
*   At least 310 grams of Carb
*   At least 30 grams of Fiber
*   At most 10 cans of each type of soup (fractional amounts are ok!)




## Setup Your Environment/Imports

In [ ]:
%matplotlib inline
from pylab import *

import shutil
import sys
import os.path
import pandas as pd

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

from pyomo.environ import *
SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'

# Data extraction

In [ ]:
# Retrieve data from wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/Table_of_food_nutrients')[12]

# Rename columns for convenience
table.columns = ["Food", "Measure", "Grams", "Calories", "Protein","Carb.","Fiber","Fat", "Sat. fat"]

# Set index
table.set_index('Food',inplace=True)

# Create dictionaries to retrieve data
my_dict = table.to_dict()
nutrients = my_dict.keys()
values = dict()
indices = list(my_dict["Measure"].keys())
for nutrient in nutrients:
  values[nutrient] = my_dict[nutrient]

In [ ]:
table


,Measure,Grams,Calories,Protein,Carb.,Fiber,Fat,Sat. fat
Food,,,,,,,,
Bean soups,1 cup,250,190,8,30,0.6,5,4
Beef and vegetable,1 cup,250,100,6,11,0.5,4,4
"Bouillon, broth, consommé",1 cup,240,24,5,0,0.0,0,0
chicken or turkey,1 cup,250,75,4,10,0.0,2,2
"Clam chowder, without milk",1 cup,255,85,5,12,0.5,2,8
"Cream soups (asparagus, celery, etc.)",1 cup,255,200,7,18,1.2,12,11
"Noodle, rice, barley",1 cup,250,115,6,13,0.2,4,3
Split-pea soup,1 cup,250,147,8,25,0.5,3,3
"Tomato soup, diluted w/milk",1 cup,245,175,6,22,0.5,7,6


In [ ]:
import pandas as pd
table["Soup_Index"]= ["S1","S2","S3",'S4','S5','S6','S7','S8','S9','S10']


**CONSTRAINS**
*   At least 2,000 Calories
*   At least 50 grams of Protein
*   At least 310 grams of Carb
*   At least 30 grams of Fiber
*   At most 10 cans of each type of soup (fractional amounts are ok!)

**Objective Function**

$\min 4S1 + 4S2 + 0S3 + 2S4 + 8S5 + 11S6 + 3S7 + 3S8 + 6S9 +2S10$ `objective function (minimizing the amount of saturated fat)`

**Write the Constraints**

subject to:
* $190S1 + 100S2 + 24S3 + 75S4 + 85S5 + 200S6 + 115S7 + 147S8 + 175S9 +80S10 \geq 2,000$ `(calories)`
* $8S1 + 6S2 + 5S3 + 4S4 + 5S5 + 7S6 + 6S7 + 8S8 + 6S9 +4S10 \geq 50$ `(protein)`
* $30S1 + 11S2 + 0S3 + 10S4 + 12S5 + 18S6 + 13S7 + 25S8 + 22S9 +14S10 \geq 310$ `(carb)`
* $0.6S1 + 0.5S2 + 0.0S3 + 0.0S4 + 0.5S5 + 1.2S6 + 0.2S7 + 0.5S8 + 0.5S9 +0.0S10 \geq 30$ `(fiber)`

* $S1, S2, S3, S4, S5, S6, S7, S8, S9, S10 \leq 10$ `(at most 10 can of each type)`
* $S1, S2, S3, S4, S5, S6, S7, S8, S9, S10 \geq 0$ `(nonnegativity)`

In [ ]:
# declare the model
model = ConcreteModel()

In [ ]:
# declare decision variables
model.S1 = Var(domain=NonNegativeReals)
model.S2 = Var(domain=NonNegativeReals)
model.S3 = Var(domain=NonNegativeReals)
model.S4 = Var(domain=NonNegativeReals)
model.S5 = Var(domain=NonNegativeReals)
model.S6 = Var(domain=NonNegativeReals)
model.S7 = Var(domain=NonNegativeReals)
model.S8 = Var(domain=NonNegativeReals)
model.S9 = Var(domain=NonNegativeReals)
model.S10 = Var(domain=NonNegativeReals)

In [ ]:
# BOUNDS

# Declare both a lower and an upper bound for your variable
model.S1 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S2 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S3 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S4 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S5 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S6 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S7 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S8 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S9 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)
model.S10 = Var(domain=NonNegativeReals,bounds=(0,10),initialize=0)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
# declare objective
obj_expr = 4*model.S1 + 4*model.S2 + 0*model.S3 + 2*model.S4 + 8*model.S5 + 11*model.S6 + 3*model.S7 + 3*model.S8 + 6*model.S9 +2*model.S10
model.sat_fat = Objective(
                      expr = obj_expr,
                      sense = minimize)

In [ ]:
# declare objeconstraints
model.Calories = Constraint(expr = 190*model.S1 + 100*model.S2 + 24*model.S3 + 75*model.S4 + 85*model.S5 + 200*model.S6 + 115*model.S7 + 147*model.S8 + 175*model.S9 +80*model.S10 >= 2000)
model.Protein = Constraint(expr = 8*model.S1 + 6*model.S2 + 5*model.S3 + 4*model.S4 + 5*model.S5 + 7*model.S6 + 6*model.S7 + 8*model.S8 + 6*model.S9 +4*model.S10 >= 50)
model.Carb = Constraint(expr = 30*model.S1 + 11*model.S2 + 0*model.S3 + 10*model.S4 + 12*model.S5 + 18*model.S6 + 13*model.S7 + 25*model.S8 + 22*model.S9 + 14*model.S10 >= 310)
model.Fiber = Constraint(expr = 0.6*model.S1 + 0.5*model.S2 + 0.0*model.S3 + 0.0*model.S4 + 0.5*model.S5 + 1.2*model.S6 + 0.2*model.S7 + 0.5*model.S8 + 0.5*model.S9 + 0.0*model.S10 >= 30)

In [ ]:
# show the model I've created
model.pprint()

In [ ]:
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 244.0
  Upper bound: 244.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 10
  Number of nonzeros: 36
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0033922195434570312
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [ ]:
# show the results
print("Sat_fat                     = ", model.sat_fat(), " per week")
print("Bean soups                  = ", model.S1(), "  cans per week")
print("Beef and vegetable          = ", model.S2(), "  cans per week")
print("Bouillon, broth, consommé   = ", model.S3(), "   cans per week")
print("chicken or turkey           = ", model.S4(), "   cans per week")
print("Clam chowder, without milk  = ", model.S5(), "   cans per week")
print("Cream soups                 = ", model.S6(), "  cans per week")
print("Noodle, rice, barley        = ", model.S7(), "   cans per week")
print("Split-pea soup              = ", model.S8(), "  cans per week")
print("omato soup, diluted w/milk  = ", model.S9(), "   cans per week")
print("Vegetable (vegetarian)      = ", model.S10(), "   cans per week")

Sat_fat                     =  244.0  per week
Bean soups                  =  10.0   cans per week
Beef and vegetable          =  10.0   cans per week
Bouillon, broth, consommé   =  0.0    cans per week
chicken or turkey           =  0.0    cans per week
Clam chowder, without milk  =  0.0    cans per week
Cream soups                 =  10.0   cans per week
Noodle, rice, barley        =  0.0    cans per week
Split-pea soup              =  10.0   cans per week
omato soup, diluted w/milk  =  4.0    cans per week
Vegetable (vegetarian)      =  0.0    cans per week


# Sensitivity Report

Use the code below to generate the sensitivity report.

In [ ]:
# First, we will save the model (you will see the file model.lp showing up on the left after executing the line below)
model.write("/content/model.lp", io_options={'symbolic_solver_labels': True})

# After running the line below, we will generate the file "sensit.sen", which contains the report we want to see
!/usr/bin/glpsol -m /content/model.lp --lp --ranges sensit.sen

# Display report
!cat /content/sensit.sen

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 -m /content/model.lp --lp --ranges sensit.sen
Reading problem data from '/content/model.lp'...
4 rows, 10 columns, 36 non-zeros
76 lines were read
GLPK Simplex Optimizer 5.0
4 rows, 10 columns, 36 non-zeros
Preprocessing...
4 rows, 10 columns, 36 non-zeros
Scaling...
 A: min|aij| =  2.000e-01  max|aij| =  2.000e+02  ratio =  1.000e+03
GM: min|aij| =  5.967e-01  max|aij| =  1.676e+00  ratio =  2.809e+00
EQ: min|aij| =  3.597e-01  max|aij| =  1.000e+00  ratio =  2.780e+00
Constructing initial basis...
Size of triangular part is 4
      0: obj =   0.000000000e+00 inf =   8.587e+01 (4)
      8: obj =   3.100000000e+02 inf =   0.000e+00 (0)
*    11: obj =   2.440000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Write sensitivity analysis report to 'sensit.sen'...
GLPK 5.0  - SENSITIVITY ANALYSIS REPORT                                                   

In [ ]:
table

,Measure,Grams,Calories,Protein,Carb.,Fiber,Fat,Sat. fat,Soup_Index
Food,,,,,,,,,
Bean soups,1 cup,250,190,8,30,0.6,5,4,S1
Beef and vegetable,1 cup,250,100,6,11,0.5,4,4,S2
"Bouillon, broth, consommé",1 cup,240,24,5,0,0.0,0,0,S3
chicken or turkey,1 cup,250,75,4,10,0.0,2,2,S4
"Clam chowder, without milk",1 cup,255,85,5,12,0.5,2,8,S5
"Cream soups (asparagus, celery, etc.)",1 cup,255,200,7,18,1.2,12,11,S6
"Noodle, rice, barley",1 cup,250,115,6,13,0.2,4,3,S7
Split-pea soup,1 cup,250,147,8,25,0.5,3,3,S8
"Tomato soup, diluted w/milk",1 cup,245,175,6,22,0.5,7,6,S9
